In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import cupy as cp # linear algebra
import cudf # data processing, CSV file I/O (e.g. cudf.read_csv)
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from cuml.metrics import roc_auc_score
import shap
import gc
from random import shuffle
from sklearn.preprocessing import LabelEncoder

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import xgboost
xgboost.__version__

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/test.csv')

In [ ]:
train.head()

In [ ]:
columns = test.columns[1:]
columns

In [ ]:
target = np.hstack([np.ones(train.shape[0]), np.zeros(test.shape[0])])

In [ ]:
train_test = np.vstack([train[columns].values, test[columns].values])

In [ ]:
train_test.shape

In [ ]:
index = list(range(train_test.shape[0]))
shuffle(index)

In [ ]:
train_test = train_test[index, :]
target = target[index]

In [ ]:
train_test = train_test.astype(np.float)

In [ ]:
train, test, y_train, y_test = train_test_split(train_test, target, test_size=0.33, random_state=42)

In [ ]:
del train_test
gc.collect()
gc.collect()

In [ ]:
train = xgboost.DMatrix(train, label=y_train)
val = xgboost.DMatrix(test, label=y_test)

In [ ]:
%%time
param = {
    'eta': 0.05,
    'max_depth': 10,
    'subsample': 0.8,
    'colsample_bytree': 0.7,
    'objective': 'reg:logistic',
    'eval_metric': 'auc',
    'tree_method': 'gpu_hist', 
    'predictor': 'gpu_predictor'
}
clf = xgboost.train(param, train, 600)

In [ ]:
preds = clf.predict(val)

In [ ]:
roc_auc_score(y_test, preds)

The AUC of 0.4987 is very close to the perfectly mixed sample, and for all practical purposes there seems to be no discernable difference between the train and test set. Nonehtless, let's take a look at what features may be the most different between the two sets.

In [ ]:
%%time
shap_preds = clf.predict(val, pred_contribs=True) 

In [ ]:
shap.summary_plot(shap_preds[:,:-1], pd.DataFrame(test, columns=columns))